# Turnstile Traffic Cleaner

- This notebook reads in the weekly turnstile data published by the MTA and calculates the number of exits per turnstile per station per day
- Data can be found [here](http://web.mta.info/developers/turnstile.html)

In [574]:
import pandas as pd
import numpy as np

In [575]:
# read in data from mta website
data1 = pd.read_csv('http://web.mta.info/developers/data/nyct/turnstile/turnstile_160206.txt')
data2 = pd.read_csv('http://web.mta.info/developers/data/nyct/turnstile/turnstile_160213.txt')
data3 = pd.read_csv('http://web.mta.info/developers/data/nyct/turnstile/turnstile_160220.txt')
data4 = pd.read_csv('http://web.mta.info/developers/data/nyct/turnstile/turnstile_160227.txt')
data5 = pd.read_csv('http://web.mta.info/developers/data/nyct/turnstile/turnstile_160305.txt')

# concatenate all into one big dataframe
data = pd.concat([data1, data2, data3, data4, data5])

In [622]:
data.columns

Index([u'C/A', u'UNIT', u'SCP', u'STATION', u'LINENAME', u'DIVISION', u'DATE',
       u'TIME', u'DESC', u'ENTRIES', u'EXITS', u'TURNSTILE'],
      dtype='object')

In [623]:
# fix stupid EXITS column heading
data.rename(columns = {'EXITS                                                               ':'EXITS','DATE':'DATE'}
            ,inplace = True)
data.head()

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS,TURNSTILE
0,A002,R051,02-00-00,59 ST,NQR456,BMT,01/30/2016,03:00:00,REGULAR,5519707,1863544,R05102-00-00
1,A002,R051,02-00-00,59 ST,NQR456,BMT,01/30/2016,07:00:00,REGULAR,5519718,1863554,R05102-00-00
2,A002,R051,02-00-00,59 ST,NQR456,BMT,01/30/2016,11:00:00,REGULAR,5519829,1863630,R05102-00-00
3,A002,R051,02-00-00,59 ST,NQR456,BMT,01/30/2016,15:00:00,REGULAR,5520132,1863693,R05102-00-00
4,A002,R051,02-00-00,59 ST,NQR456,BMT,01/30/2016,19:00:00,REGULAR,5520596,1863760,R05102-00-00


In [624]:
# create a unique identifier for each turnstile at a station
data['TURNSTILE'] = data['UNIT'] + data['SCP']

In [625]:
# look at stations on a given line to make sure we're spelling them right
data['STATION'][data['LINENAME']=='GL'].unique()

array(['METROPOLITAN AV'], dtype=object)

In [626]:
# create variable for our station:
st = 'METROPOLITAN AV'

# create a new dataframe with only target station data
station = data[data['STATION']==st]
# need to add additional line name criterion for metropolitan av since there are two stations with that name
station = data[(data['STATION']=='METROPOLITAN AV') & (data['LINENAME']=='GL')] 
station.head()

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS,TURNSTILE
90385,N409,R268,00-00-00,METROPOLITAN AV,GL,IND,01/30/2016,00:00:00,REGULAR,4224935,9758579,R26800-00-00
90386,N409,R268,00-00-00,METROPOLITAN AV,GL,IND,01/30/2016,04:00:00,REGULAR,4225039,9758791,R26800-00-00
90387,N409,R268,00-00-00,METROPOLITAN AV,GL,IND,01/30/2016,08:00:00,REGULAR,4225061,9758842,R26800-00-00
90388,N409,R268,00-00-00,METROPOLITAN AV,GL,IND,01/30/2016,12:00:00,REGULAR,4225167,9758970,R26800-00-00
90389,N409,R268,00-00-00,METROPOLITAN AV,GL,IND,01/30/2016,16:00:00,REGULAR,4225359,9759386,R26800-00-00


In [627]:
# create lists of turnstiles and dates; these will become our column and index values
turn_list = np.asarray(station.TURNSTILE.unique())
date_list = np.asarray(station.DATE.unique())
print 'Number of turnstiles:',len(turn_list)
print turn_list
print ''
print '- - '*20
print ''
print 'Days of the week:',len(date_list)
print date_list

Number of turnstiles: 7
['R26800-00-00' 'R26800-00-01' 'R26800-00-02' 'R26800-03-00' 'R26800-03-01'
 'R26800-06-00' 'R26800-06-01']

- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 

Days of the week: 34
['01/30/2016' '01/31/2016' '02/01/2016' '02/02/2016' '02/03/2016'
 '02/04/2016' '02/05/2016' '02/06/2016' '02/07/2016' '02/08/2016'
 '02/09/2016' '02/10/2016' '02/11/2016' '02/13/2016' '02/14/2016'
 '02/15/2016' '02/16/2016' '02/17/2016' '02/18/2016' '02/19/2016'
 '02/20/2016' '02/21/2016' '02/22/2016' '02/23/2016' '02/24/2016'
 '02/25/2016' '02/26/2016' '02/27/2016' '02/28/2016' '02/29/2016'
 '03/01/2016' '03/02/2016' '03/03/2016' '03/04/2016']


### This assumes that the number of people passing through a turnstile on a given day is the minimum (beginning of the day) minus the maximum (end of the day) value

In [628]:
my_list = []
for d in date_list:
    for i in turn_list:
        try:
            t_max = max(station.EXITS[(station['TURNSTILE']==i) & (station['DATE']==d)]) 
            t_min = min(station.EXITS[(station['TURNSTILE']==i) & (station['DATE']==d)])
            out = t_max - t_min
            my_list.append(out)
        except ValueError: # adding exception handling for metropolitan ave issue
            my_list.append(out)

In [629]:
# need to reshape the array since the my_list output is a Nx1 array, but we want 7xN
# use len() to account for different numbers of days and turnstiles
turn_array = np.asarray(my_list).reshape(len(date_list),len(turn_list))
turn_list = list(station.TURNSTILE.unique()) # convert to list for easier dataframe creation

In [630]:
final = pd.DataFrame(turn_array,index = date_list, columns = turn_list).T # transpose to make it more intelligible
final

,01/30/2016,01/31/2016,02/01/2016,02/02/2016,02/03/2016,02/04/2016,02/05/2016,02/06/2016,02/07/2016,02/08/2016,...,02/24/2016,02/25/2016,02/26/2016,02/27/2016,02/28/2016,02/29/2016,03/01/2016,03/02/2016,03/03/2016,03/04/2016
R26800-00-00,1495,1298,1298,1305,1329,1354,1397,1455,1213,1234,...,1201,1365,1317,1294,1100,1364,1427,1322,1396,1426
R26800-00-01,728,590,506,546,524,555,646,787,696,508,...,574,597,583,593,593,570,685,565,640,678
R26800-00-02,373,271,238,251,245,225,269,391,351,202,...,272,243,293,335,287,215,475,252,416,489
R26800-03-00,404,327,299,302,308,315,358,365,299,264,...,324,383,351,342,335,320,319,286,345,300
R26800-03-01,861,810,722,717,713,780,793,906,754,646,...,734,679,778,855,798,715,712,782,779,769
R26800-06-00,257,249,132,161,175,166,242,249,218,148,...,277,184,192,183,186,183,185,211,241,187
R26800-06-01,752,682,506,592,515,555,654,734,683,450,...,725,610,689,585,564,525,691,645,590,589


## Don't forget to change file names below
- just used this to save to the USI folder on my machine

In [631]:
final.to_csv('USI/station_metropolitan.csv')# changed file names manually on purpose
print 'Done!'

Done!
